In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.naive_bayes import MultinomialNB
import re

In [17]:
users = pd.read_csv('data/full/train.csv')

In [18]:
users = users[['user_id','target']]

In [5]:
tweets = pd.concat([pd.read_csv('data/propaganda/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/porn/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/spam/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/fake_followers/tweets.csv', sep='\t')[['user_id','full_text']], \
          pd.read_csv('data/genuine/tweets.csv', sep='\t')[['user_id','full_text']]])

/Users/Lorenzo/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/Lorenzo/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/Lorenzo/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,4,8,11,18,19,20,23,26) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/Lorenzo/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,8,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
 

In [6]:
tweets = tweets.drop_duplicates()

In [7]:
tweets = pd.merge(tweets, users, on='user_id')

In [8]:
tweets.drop(['user_id'], axis=1, inplace=True)

In [19]:
tweets.head()

,full_text,target
0,Tweet this link out if you want to help impli...,1
1,So much for party labels. With Democrats like...,1
2,Trump just said he’s refusing to sign a joint...,1
3,When an American President attacks our closes...,1
4,TRUMP disgraced the Presidency and the United...,1


In [20]:
tweets['full_text'] = tweets['full_text'].apply(lambda x: x.replace('RT', ''))

In [11]:
def remove_rt(x):
    
    try:
        return x[x.rindex(':')+1:]
    except:
        return x

In [12]:
tweets['full_text'] = tweets['full_text'].apply(lambda x: remove_rt(x))

In [13]:
tweets['full_text'] = tweets['full_text'].astype(str)

In [22]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(tweets.full_text)
labels = tweets.target
features.shape

(1285631, 217159)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(tweets['full_text'], tweets['target'], random_state = 0)
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [24]:
y_pred = clf.predict(X_test[:-1])

ValueError: could not convert string to float: '@SalahTheMan "It is not because things are difficult that we do not dare, it is because we do not dare that they are difficult." - Seneca'

In [36]:
X_test.loc[1191716]

'This axe body wash have me smelling like a snack tho 😁'

In [55]:
type(tweets.iloc[1191716]['full_text'])

str

SyntaxError: invalid character in identifier (<ipython-input-49-eb78a41c5af4>, line 1)